In [1]:
# import xlsxwriter
import pylightxl as xl
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle

In [2]:
# readxl returns a pylightxl database that holds all worksheets and its data
db = xl.readxl(fn='SPX_train_0.xlsx')

In [3]:
db.ws_names

['Sheet1']

In [4]:
db.ws(ws='Sheet1').row(row=1)

['Time',
 'Returns',
 'dp',
 'dy',
 'ep',
 'de',
 'svar',
 'bm',
 'ntis',
 'tbl',
 'lty',
 'ltr',
 'tms',
 'dfy',
 'dfr',
 'infl']

In [5]:
file_rows = []

for row in db.ws(ws='Sheet1').rows:
       file_rows.append(row)


In [6]:
df = pd.DataFrame(file_rows[1:])
df.columns = file_rows[0]
df.head()

,Time,Returns,dp,dy,ep,de,svar,bm,ntis,tbl,lty,ltr,tms,dfy,dfr,infl
0,1945-01,1,-3.041609,-3.027403,-2.662340,-0.379269,0.000924,0.735342,0.016454,0.0038,0.0240,0.0127,0.0202,0.0077,-0.0051,0.000000
1,1945-02,0,-3.096132,-3.036338,-2.711553,-0.384579,0.000655,0.704489,0.014836,0.0038,0.0236,0.0077,0.0198,0.0076,-0.0031,0.000000
2,1945-03,1,-3.043790,-3.091042,-2.653829,-0.389961,0.001887,0.767883,0.015963,0.0038,0.0236,0.0021,0.0198,0.0076,-0.0003,0.000000
3,1945-04,1,-3.128109,-3.043790,-2.724389,-0.403720,0.001398,0.715063,0.015086,0.0038,0.0228,0.0160,0.0190,0.0075,-0.0142,0.000000
4,1945-05,0,-3.139500,-3.128109,-2.722106,-0.417394,0.000921,0.702911,0.019773,0.0038,0.0226,0.0056,0.0188,0.0070,-0.0067,0.005618


In [7]:
corr_df = df.corr()
corr_df

,Returns,dp,dy,ep,de,svar,bm,ntis,tbl,lty,ltr,tms,dfy,dfr,infl
Returns,1.000000,-0.013558,-0.010382,-0.024331,0.016652,-0.032973,-0.062108,-0.070254,-0.108139,-0.091426,0.042334,0.055445,0.006196,0.032814,-0.124365
dp,-0.013558,1.000000,0.995444,0.784057,0.316316,-0.067223,0.893108,0.321757,0.259010,0.172625,-0.015995,-0.231585,0.090029,-0.001723,0.190462
dy,-0.010382,0.995444,1.000000,0.780035,0.315562,-0.099024,0.887306,0.317785,0.254149,0.169771,-0.008529,-0.226418,0.091557,0.018966,0.182688
ep,-0.024331,0.784057,0.780035,1.000000,-0.340809,-0.168505,0.819701,0.244682,0.364006,0.236346,0.001676,-0.338798,-0.028332,-0.070064,0.217077
de,0.016652,0.316316,0.315562,-0.340809,1.000000,0.155721,0.099933,0.113382,-0.164032,-0.099761,-0.026788,0.167043,0.179666,0.104476,-0.043292
svar,-0.032973,-0.067223,-0.099024,-0.168505,0.155721,1.000000,-0.085317,-0.194539,-0.040499,0.020425,0.143990,0.137262,0.309874,-0.140850,-0.095177
bm,-0.062108,0.893108,0.887306,0.819701,0.099933,-0.085317,1.000000,0.329714,0.431686,0.338215,-0.006183,-0.278345,0.221486,-0.007548,0.289631
ntis,-0.070254,0.321757,0.317785,0.244682,0.113382,-0.194539,0.329714,1.000000,0.029686,-0.021671,-0.078767,-0.114891,-0.376040,0.012828,0.136347
tbl,-0.108139,0.259010,0.254149,0.364006,-0.164032,-0.040499,0.431686,0.029686,1.000000,0.902038,0.042228,-0.392109,0.362053,-0.037519,0.294536
lty,-0.091426,0.172625,0.169771,0.236346,-0.099761,0.020425,0.338215,-0.021671,0.902038,1.000000,0.043965,0.043392,0.515961,-0.003863,0.239608


In [8]:
returns = corr_df["Returns"]
returns

Returns    1.000000
dp        -0.013558
dy        -0.010382
ep        -0.024331
de         0.016652
svar      -0.032973
bm        -0.062108
ntis      -0.070254
tbl       -0.108139
lty       -0.091426
ltr        0.042334
tms        0.055445
dfy        0.006196
dfr        0.032814
infl      -0.124365
Name: Returns, dtype: float64

In [9]:
five_percent_corr = returns[abs(returns) > 0.05 ]

In [10]:
five_percent_corr

Returns    1.000000
bm        -0.062108
ntis      -0.070254
tbl       -0.108139
lty       -0.091426
tms        0.055445
infl      -0.124365
Name: Returns, dtype: float64

In [11]:
X = df[[c for c in df.columns if c in five_percent_corr]]
y = df["Returns"]

In [12]:
X.head()

,Returns,bm,ntis,tbl,lty,tms,infl
0,1,0.735342,0.016454,0.0038,0.0240,0.0202,0.000000
1,0,0.704489,0.014836,0.0038,0.0236,0.0198,0.000000
2,1,0.767883,0.015963,0.0038,0.0236,0.0198,0.000000
3,1,0.715063,0.015086,0.0038,0.0228,0.0190,0.000000
4,0,0.702911,0.019773,0.0038,0.0226,0.0188,0.005618


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [14]:
lr = LogisticRegression()

lr.fit(X_train, y_train)


LogisticRegression()

In [15]:
y_pred = lr.predict(X_test)

In [16]:
confusion_matrix(y_test, y_pred)

array([[ 79,   0],
       [  0, 101]])

In [17]:
filename = "predict_now.sav"
pickle.dump(lr, open(filename, 'wb'))